## Проект № 6 "Выбираем авто выгодно"

### Ноутбук для парсинга данных по автомашинам с сайта auto.ru, с целью получения обущающего датасета для построения модели предсказания стоимости автомобиля

In [1]:
# загружаем необходимые библиотеки для получения и обработки данных
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
pd.set_option('display.max_columns', None)
import csv
import json
from tqdm import tqdm
import numpy as np

Процесс парсинга данных с сайта был разделен на 2 этапа. На первом этапе с сайта ссылки на объявления по продаже автомобилей. На втором этапе переходили по полученным ссылкам и собирали данные об автомобиле из объявления.

### 1. Сбор ссылок на объявления

На этом этапе ссылки на объявления собирались по 12 маркам автомобилей, имеющимся в тестовом датасете.

In [2]:
# функция для получения всех ссылок на объявления по машинам выбранных марок
def get_all_links(url_list):
    # создадим пустой список, куда будем добавлять ссылки на объявления
    all_car = []
    city = url_list[0][16:url_list[0].index('/', 17)]
    # пройдемся по каждой выбранной марке
    for url in url_list:
        # получим количество объявлений по выбранной марке авто
        res = requests.get(url)
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text, 'html.parser')
        number_of_ads = soup.find('button',
                                  class_='Button Button_color_blue Button_size_m Button_type_button Button_width_full')\
                            .find('span', class_='ButtonWithLoader__content').text
        number_of_ads = number_of_ads.replace('\xa0', '')
        number_of_ads = int(re.findall('\d+', number_of_ads)[0])
        # на одной странице 38 объявлений
        number_of_pages = int(number_of_ads / 38)
        # максимальное количество страниц, которые можно посмотреть равно 99
        # чтобы не копировать одинаковые ссылки, будем смотреть первые 99 страниц (если их больше)
        if number_of_pages > 99:
            number_of_pages = 99
        # пройдемся по каждой странице выбранной марки
        for i in range(1, number_of_pages+1):
            # получаем и сохраняем ссылки по каждой машине
            url_page = url + '/?page=' + str(i)
            res = requests.get(url_page)
            res.encoding = 'utf-8'
            # задержка времени при отправке запросов
            time.sleep(0.1)
            soup = BeautifulSoup(res.text, 'html.parser')
            car_page = soup.find_all('a', class_='Link ListingItemTitle__link')
            car_list = list(map(lambda x: x.get('href'), car_page))
            for row in car_list:
                all_car.append(row)
        brand = re.findall('\w+', url[22+len(city):])[0]
        print(f'Все объявления по марке {brand} в городе {city} собраны')
    return all_car


def get_all_links_in_city(city):
    # сгенерируем список основного сайта по маркам авто по городу
    url_list = ['https://auto.ru/'+city+'/cars/'+brand+'/used/' for brand in brands]
    all_car = get_all_links(url_list)
    # сохраним в отдельный файл
    with open(f'{city}_all_car_11_11_2021.csv', 'a') as myfile:
        wr = csv.writer(myfile)
        for row in all_car:
            wr.writerow([row])
    df = pd.read_csv(f'{city}_all_car_11_11_2021.csv', header=None)
    print(f'По городу {city} собрано {df.shape[0]} ссылок')
    return df

In [3]:
# все нужные нам марки автомобилей соберем в один список
brands = ['honda', 'skoda', 'audi', 'volvo', 'bmw', 'nissan', 'infiniti',
          'mercedes', 'toyota', 'lexus', 'volkswagen', 'mitsubishi']

Так как объявлений о продаже в данный момент немного (по Москве менее 30 тыс.), соберем все объявления по городам-миллионикам.

In [4]:
# список городов-миллионников как на сайте auto.ru
cities = ['moskva', 'sankt-peterburg', 'novosibirsk', 'ekaterinburg', 'kazan',
         'nizhniy_novgorod', 'chelyabinsk', 'samara', 'omsk', 'rostov-na-donu',
         'ufa', 'krasnoyarsk', 'voronezh', 'perm', 'volgograd']

In [5]:
for city in cities:
    get_all_links_in_city(city)

Все объявления по марке honda в городе moskva собраны
Все объявления по марке skoda в городе moskva собраны
Все объявления по марке audi в городе moskva собраны
Все объявления по марке volvo в городе moskva собраны
Все объявления по марке bmw в городе moskva собраны
Все объявления по марке nissan в городе moskva собраны
Все объявления по марке infiniti в городе moskva собраны
Все объявления по марке mercedes в городе moskva собраны
Все объявления по марке toyota в городе moskva собраны
Все объявления по марке lexus в городе moskva собраны
Все объявления по марке volkswagen в городе moskva собраны
Все объявления по марке mitsubishi в городе moskva собраны
По городу moskva собрано 28200 ссылок
Все объявления по марке honda в городе sankt-peterburg собраны
Все объявления по марке skoda в городе sankt-peterburg собраны
Все объявления по марке audi в городе sankt-peterburg собраны
Все объявления по марке volvo в городе sankt-peterburg собраны
Все объявления по марке bmw в городе sankt-peter

Все объявления по марке skoda в городе krasnoyarsk собраны
Все объявления по марке audi в городе krasnoyarsk собраны
Все объявления по марке volvo в городе krasnoyarsk собраны
Все объявления по марке bmw в городе krasnoyarsk собраны
Все объявления по марке nissan в городе krasnoyarsk собраны
Все объявления по марке infiniti в городе krasnoyarsk собраны
Все объявления по марке mercedes в городе krasnoyarsk собраны
Все объявления по марке toyota в городе krasnoyarsk собраны
Все объявления по марке lexus в городе krasnoyarsk собраны
Все объявления по марке volkswagen в городе krasnoyarsk собраны
Все объявления по марке mitsubishi в городе krasnoyarsk собраны
По городу krasnoyarsk собрано 4414 ссылок
Все объявления по марке honda в городе voronezh собраны
Все объявления по марке skoda в городе voronezh собраны
Все объявления по марке audi в городе voronezh собраны
Все объявления по марке volvo в городе voronezh собраны
Все объявления по марке bmw в городе voronezh собраны
Все объявления по

### 2. Получение данных о продаваемом автомобиле из объявления

In [6]:
# функция для сбора данных по ссылке на объявление
def get_car_data(links):
    
    # создаем DataFrame для данных
    df_train = pd.DataFrame()
    
    for url in tqdm(links):
        
        try:
            res = requests.get(url)
            res.encoding = 'utf-8'
            soup = BeautifulSoup(res.text, 'html.parser')
            
            script1 = soup.find('script', type='application/ld+json').contents[0]
            script1_json = json.loads(script1)
            script2 = soup.find('script', id='initial-state').contents[0]
            script2_json = json.loads(script2)
            ul = soup.find('ul', class_='CardInfo')
        except:
            continue
        
        # создадим пустой словарь для данных одного авто
        train = {}
        
        try:
            train['bodyType'] = script1_json['bodyType']
        except:
            np.nan
        
        try:
            train['brand'] = script1_json['brand']
        except:
            np.nan
            
        try:
            train['car_url'] = script1_json['offers']['url']
        except:
            np.nan
        
        try:
            train['color'] = script1_json['color']
        except:
            np.nan
            
        try:
            train['complectation_dict'] = np.nan
        except:
            np.nan
        
        try:
            train['description'] = script1_json['description']
        except:
            np.nan
        
        try:
            train['engineDisplacement'] = script1_json['vehicleEngine']['engineDisplacement']
        except:
            np.nan
        
        try:
            train['enginePower'] = script1_json['vehicleEngine']['enginePower']
        except:
            np.nan
        
        try:
            train['equipment_dict'] = str(script2_json['card']['vehicle_info']['equipment'])
        except:
            np.nan
        
        try:
            train['fuelType'] = script1_json['vehicleEngine']['fuelType']
        except:
            np.nan
        
        try:
            train['image'] = script1_json['image']
        except:
            np.nan
        
        try:
            train['mileage'] = script2_json['card']['state']['mileage']
        except:
            np.nan
        
        try:
            train['modelDate'] = script1_json['modelDate']
        except:
            np.nan
        
        try:
            train['model_info'] = str(script2_json['card']['vehicle_info']['model_info'])
        except:
            np.nan
        
        try:
            train['model_name'] = script2_json['card']['vehicle_info']['model_info']['code']
        except:
            np.nan
            
        try:
            train['name'] = script2_json['card']['vehicle_info']['tech_param']['human_name']
        except:
            np.nan
        
        try:
            train['numberOfDoors'] = script1_json['numberOfDoors']
        except:
            np.nan
            
        try:
            train['parsing_unixtime'] = int(time.time())
        except:
            np.nan
            
        try:
            train['priceCurrency'] = script1_json['offers']['priceCurrency']
        except:
            np.nan
        
        try:
            train['productionDate'] = script1_json['productionDate']
        except:
            np.nan
            
        try:
            train['sell_id'] = script2_json['card']['id']
        except:
            np.nan
        
        try:
            train['super_gen'] = str(script2_json['card']['vehicle_info']['tech_param'])
        except:
            np.nan
            
        try:
            train['vehicleConfiguration'] = script1_json['vehicleConfiguration']
        except:
            np.nan
        
        try:
            train['vehicleTransmission'] = script1_json['vehicleTransmission']
        except:
            np.nan
        
        try:
            train['vendor'] = np.nan
        except:
            np.nan
        
        try:
            train['Владельцы'] = ul.find_all('li', class_='CardInfoRow_ownersCount')[0].find_all('span')[1].text
        except:
            np.nan
        
        try:
            train['Владение'] = ul.find_all('li', class_='CardInfoRow_owningTime')[0].find_all('span')[1].text
        except:
            np.nan
        
        try:
            train['ПТС'] = ul.find_all('li', class_='CardInfoRow_pts')[0].find_all('span')[1].text
        except:
            np.nan
            
        try:
            train['Привод'] = ul.find_all('li', class_='CardInfoRow_drive')[0].find_all('span')[1].text
        except:
            np.nan
            
        try:
            train['Руль'] = ul.find_all('li', class_='CardInfoRow_wheel')[0].find_all('span')[1].text
        except:
            np.nan
        
        try:
            train['Состояние'] = ul.find_all('li', class_='CardInfoRow_state')[0].find_all('span')[1].text
        except:
            np.nan
        
        try:
            train['Таможня'] = ul.find_all('li', class_='CardInfoRow_customs')[0].find_all('span')[1].text
        except:
            np.nan
        
        try:
            train['price'] = script1_json['offers']['price']
        except:
            np.nan
            
        
        df_train = df_train.append([train])
    return df_train

In [7]:
df = pd.read_csv('kazan_all_car_11_11_2021.csv', header=None)
df

,0
0,https://auto.ru/cars/used/sale/honda/accord/11...
1,https://auto.ru/cars/used/sale/honda/civic/110...
2,https://auto.ru/cars/used/sale/honda/cr_v/1105...
3,https://auto.ru/cars/used/sale/honda/civic/110...
4,https://auto.ru/cars/used/sale/honda/accord/11...
...,...
5188,https://auto.ru/cars/used/sale/mitsubishi/outl...
5189,https://auto.ru/cars/used/sale/mitsubishi/paje...
5190,https://auto.ru/cars/used/sale/mitsubishi/spac...
5191,https://auto.ru/cars/used/sale/mitsubishi/paje...


In [8]:
# проверим на дубликаты
df.duplicated().sum()

863

In [9]:
# удаляем дубликаты
df = df.drop_duplicates()
df.duplicated().sum()

0

In [10]:
# получим список ссылок на сайты
all_car = df[0].tolist()

In [14]:
# запускаем сбор данных
df_train_kazan = get_car_data(all_car)

100%|██████████| 4330/4330 [1:17:26<00:00,  1.07s/it]


In [15]:
df_train_kazan

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,седан,HONDA,https://auto.ru/cars/used/sale/honda/accord/11...,серый,NaN,ПРОДАЮ HONDA ACCORD 2020 г.в.\nПРАКТИЧЕСКИЙ Н...,2.0 LTR,252 N12,"{'engine-proof': True, 'tinted-glass': True, '...",бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,11000,2017,"{'code': 'ACCORD', 'name': 'Accord', 'ru_name'...",ACCORD,2.0 MT (252 л.с.),4,1636613967,RUB,2020,1105857897,"{'id': '21121653', 'name': '', 'nameplate': ''...",SEDAN MECHANICAL 2.0,механическая,NaN,1 владелец,1 год и 5 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,2400000.0
0,хэтчбек 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/civic/110...,серый,NaN,Honda Civic куплен в феврале 2009 года у офици...,2.2 LTR,140 N12,"{'cruise-control': True, 'asr': True, 'start-s...",дизель,https://avatars.mds.yandex.net/get-autoru-vos/...,209000,2008,"{'code': 'CIVIC', 'name': 'Civic', 'ru_name': ...",CIVIC,2.2d MT (140 л.с.),5,1636613969,RUB,2008,1105732996,"{'id': '7699571', 'name': '', 'nameplate': '',...",HATCHBACK_5_DOORS MECHANICAL 2.2,механическая,NaN,2 владельца,12 лет и 9 месяцев,Дубликат,передний,Левый,Не требует ремонта,Растаможен,650000.0
0,внедорожник 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/cr_v/1105...,чёрный,NaN,Отличный семейный автомобиль,2.2 LTR,140 N12,"{'cruise-control': True, 'engine-proof': True,...",дизель,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,209000,2009,"{'code': 'CR_V', 'name': 'CR-V', 'ru_name': 'C...",CR_V,2.2d AT (140 л.с.) 4WD,5,1636613970,RUB,2011,1105543170,"{'id': '20392406', 'name': '', 'nameplate': ''...",ALLROAD_5_DOORS AUTOMATIC 2.2,автоматическая,NaN,3 или более,3 года и 11 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1170000.0
0,седан,HONDA,https://auto.ru/cars/used/sale/honda/civic/110...,чёрный,NaN,-без ДТП\n-в отличном состоянии\n-менял сцепле...,1.8 LTR,140 N12,"{'asr': True, 'tinted-glass': True, 'airbag-dr...",бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,284000,2005,"{'code': 'CIVIC', 'name': 'Civic', 'ru_name': ...",CIVIC,1.8 AT (140 л.с.),4,1636613971,RUB,2008,1105839313,"{'id': '2306779', 'name': '', 'nameplate': '',...",SEDAN AUTOMATIC 1.8,автоматическая,NaN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,650000.0
0,седан,HONDA,https://auto.ru/cars/used/sale/honda/accord/11...,чёрный,NaN,⚡ 15 000 СКИДКА В КРЕДИТ\n⚡ 15 000 СКИДКА ПРИ ...,2.2 LTR,150 N12,"{'tinted-glass': True, 'usb': True, 'windclean...",дизель,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,165000,2007,"{'code': 'ACCORD', 'name': 'Accord', 'ru_name'...",ACCORD,2.2d AT (150 л.с.),4,1636613972,RUB,2008,1105911127,"{'id': '20501294', 'name': '', 'nameplate': ''...",SEDAN AUTOMATIC 2.2,автоматическая,NaN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,920000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,внедорожник 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/outl...,серый,NaN,Есть рыжики по кузову болезнь мицубы к сожален...,2.4 LTR,160 N12,{},бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,289000,2002,"{'code': 'OUTLANDER', 'name': 'Outlander', 'ru...",OUTLANDER,2.4 MT (160 л.с.) 4WD,5,1636618608,RUB,2004,1105835713,"{'id': '2308032', 'name': '', 'nameplate': '',...",ALLROAD_5_DOORS MECHANICAL 2.4,механическая,NaN,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,500000.0
0,внедорожник 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/paje...,чёрный,NaN,Продаю легендарный внедорожник в максимальной ...,3.2 LTR,170 N12,"{'cruise-control': True, 'tinted-glass': True,...",дизель,

In [16]:
# добавим признак города
df_train_kazan['city'] = 'Казань'
df_train_kazan

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price,city
0,седан,HONDA,https://auto.ru/cars/used/sale/honda/accord/11...,серый,NaN,ПРОДАЮ HONDA ACCORD 2020 г.в.\nПРАКТИЧЕСКИЙ Н...,2.0 LTR,252 N12,"{'engine-proof': True, 'tinted-glass': True, '...",бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,11000,2017,"{'code': 'ACCORD', 'name': 'Accord', 'ru_name'...",ACCORD,2.0 MT (252 л.с.),4,1636613967,RUB,2020,1105857897,"{'id': '21121653', 'name': '', 'nameplate': ''...",SEDAN MECHANICAL 2.0,механическая,NaN,1 владелец,1 год и 5 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,2400000.0,Казань
0,хэтчбек 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/civic/110...,серый,NaN,Honda Civic куплен в феврале 2009 года у офици...,2.2 LTR,140 N12,"{'cruise-control': True, 'asr': True, 'start-s...",дизель,https://avatars.mds.yandex.net/get-autoru-vos/...,209000,2008,"{'code': 'CIVIC', 'name': 'Civic', 'ru_name': ...",CIVIC,2.2d MT (140 л.с.),5,1636613969,RUB,2008,1105732996,"{'id': '7699571', 'name': '', 'nameplate': '',...",HATCHBACK_5_DOORS MECHANICAL 2.2,механическая,NaN,2 владельца,12 лет и 9 месяцев,Дубликат,передний,Левый,Не требует ремонта,Растаможен,650000.0,Казань
0,внедорожник 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/cr_v/1105...,чёрный,NaN,Отличный семейный автомобиль,2.2 LTR,140 N12,"{'cruise-control': True, 'engine-proof': True,...",дизель,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,209000,2009,"{'code': 'CR_V', 'name': 'CR-V', 'ru_name': 'C...",CR_V,2.2d AT (140 л.с.) 4WD,5,1636613970,RUB,2011,1105543170,"{'id': '20392406', 'name': '', 'nameplate': ''...",ALLROAD_5_DOORS AUTOMATIC 2.2,автоматическая,NaN,3 или более,3 года и 11 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1170000.0,Казань
0,седан,HONDA,https://auto.ru/cars/used/sale/honda/civic/110...,чёрный,NaN,-без ДТП\n-в отличном состоянии\n-менял сцепле...,1.8 LTR,140 N12,"{'asr': True, 'tinted-glass': True, 'airbag-dr...",бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,284000,2005,"{'code': 'CIVIC', 'name': 'Civic', 'ru_name': ...",CIVIC,1.8 AT (140 л.с.),4,1636613971,RUB,2008,1105839313,"{'id': '2306779', 'name': '', 'nameplate': '',...",SEDAN AUTOMATIC 1.8,автоматическая,NaN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,650000.0,Казань
0,седан,HONDA,https://auto.ru/cars/used/sale/honda/accord/11...,чёрный,NaN,⚡ 15 000 СКИДКА В КРЕДИТ\n⚡ 15 000 СКИДКА ПРИ ...,2.2 LTR,150 N12,"{'tinted-glass': True, 'usb': True, 'windclean...",дизель,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,165000,2007,"{'code': 'ACCORD', 'name': 'Accord', 'ru_name'...",ACCORD,2.2d AT (150 л.с.),4,1636613972,RUB,2008,1105911127,"{'id': '20501294', 'name': '', 'nameplate': ''...",SEDAN AUTOMATIC 2.2,автоматическая,NaN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,920000.0,Казань
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,внедорожник 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/outl...,серый,NaN,Есть рыжики по кузову болезнь мицубы к сожален...,2.4 LTR,160 N12,{},бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,289000,2002,"{'code': 'OUTLANDER', 'name': 'Outlander', 'ru...",OUTLANDER,2.4 MT (160 л.с.) 4WD,5,1636618608,RUB,2004,1105835713,"{'id': '2308032', 'name': '', 'nameplate': '',...",ALLROAD_5_DOORS MECHANICAL 2.4,механическая,NaN,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,500000.0,Казань
0,внедорожник 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/paje...,чёрный,NaN,Продаю легендарный внедорожник в максимальной ...,3.2 LTR,170 N12,"{'cruis

In [17]:
# сохраним датафрейм
df_train_kazan.to_csv('df_train_kazan.csv')
data = pd.read_csv('df_train_kazan.csv')
data

,Unnamed: 0,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price,city
0,0,седан,HONDA,https://auto.ru/cars/used/sale/honda/accord/11...,серый,NaN,ПРОДАЮ HONDA ACCORD 2020 г.в.\nПРАКТИЧЕСКИЙ Н...,2.0 LTR,252 N12,"{'engine-proof': True, 'tinted-glass': True, '...",бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,11000,2017,"{'code': 'ACCORD', 'name': 'Accord', 'ru_name'...",ACCORD,2.0 MT (252 л.с.),4,1636613967,RUB,2020,1105857897,"{'id': '21121653', 'name': '', 'nameplate': ''...",SEDAN MECHANICAL 2.0,механическая,NaN,1 владелец,1 год и 5 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,2400000.0,Казань
1,0,хэтчбек 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/civic/110...,серый,NaN,Honda Civic куплен в феврале 2009 года у офици...,2.2 LTR,140 N12,"{'cruise-control': True, 'asr': True, 'start-s...",дизель,https://avatars.mds.yandex.net/get-autoru-vos/...,209000,2008,"{'code': 'CIVIC', 'name': 'Civic', 'ru_name': ...",CIVIC,2.2d MT (140 л.с.),5,1636613969,RUB,2008,1105732996,"{'id': '7699571', 'name': '', 'nameplate': '',...",HATCHBACK_5_DOORS MECHANICAL 2.2,механическая,NaN,2 владельца,12 лет и 9 месяцев,Дубликат,передний,Левый,Не требует ремонта,Растаможен,650000.0,Казань
2,0,внедорожник 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/cr_v/1105...,чёрный,NaN,Отличный семейный автомобиль,2.2 LTR,140 N12,"{'cruise-control': True, 'engine-proof': True,...",дизель,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,209000,2009,"{'code': 'CR_V', 'name': 'CR-V', 'ru_name': 'C...",CR_V,2.2d AT (140 л.с.) 4WD,5,1636613970,RUB,2011,1105543170,"{'id': '20392406', 'name': '', 'nameplate': ''...",ALLROAD_5_DOORS AUTOMATIC 2.2,автоматическая,NaN,3 или более,3 года и 11 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1170000.0,Казань
3,0,седан,HONDA,https://auto.ru/cars/used/sale/honda/civic/110...,чёрный,NaN,-без ДТП\n-в отличном состоянии\n-менял сцепле...,1.8 LTR,140 N12,"{'asr': True, 'tinted-glass': True, 'airbag-dr...",бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,284000,2005,"{'code': 'CIVIC', 'name': 'Civic', 'ru_name': ...",CIVIC,1.8 AT (140 л.с.),4,1636613971,RUB,2008,1105839313,"{'id': '2306779', 'name': '', 'nameplate': '',...",SEDAN AUTOMATIC 1.8,автоматическая,NaN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,650000.0,Казань
4,0,седан,HONDA,https://auto.ru/cars/used/sale/honda/accord/11...,чёрный,NaN,⚡ 15 000 СКИДКА В КРЕДИТ\n⚡ 15 000 СКИДКА ПРИ ...,2.2 LTR,150 N12,"{'tinted-glass': True, 'usb': True, 'windclean...",дизель,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,165000,2007,"{'code': 'ACCORD', 'name': 'Accord', 'ru_name'...",ACCORD,2.2d AT (150 л.с.),4,1636613972,RUB,2008,1105911127,"{'id': '20501294', 'name': '', 'nameplate': ''...",SEDAN AUTOMATIC 2.2,автоматическая,NaN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,920000.0,Казань
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4325,0,внедорожник 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/outl...,серый,NaN,Есть рыжики по кузову болезнь мицубы к сожален...,2.4 LTR,160 N12,{},бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,289000,2002,"{'code': 'OUTLANDER', 'name': 'Outlander', 'ru...",OUTLANDER,2.4 MT (160 л.с.) 4WD,5,1636618608,RUB,2004,1105835713,"{'id': '2308032', 'name': '', 'nameplate': '',...",ALLROAD_5_DOORS MECHANICAL 2.4,механическая,NaN,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,500000.0,Казань
4326,0,внедорожник 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/paje...,чёрный,NaN,Продаю легендарный внедорожник в максим

In [18]:
df = pd.read_csv('voronezh_all_car_11_11_2021.csv', header=None)
df

,0
0,https://auto.ru/cars/used/sale/honda/civic/110...
1,https://auto.ru/cars/used/sale/honda/cr_v/1105...
2,https://auto.ru/cars/used/sale/honda/stepwagon...
3,https://auto.ru/cars/used/sale/honda/cr_v/1105...
4,https://auto.ru/cars/used/sale/honda/civic/110...
...,...
3221,https://auto.ru/cars/used/sale/mitsubishi/spac...
3222,https://auto.ru/cars/used/sale/mitsubishi/cari...
3223,https://auto.ru/cars/used/sale/mitsubishi/outl...
3224,https://auto.ru/cars/used/sale/mitsubishi/cari...


In [19]:
# проверим на дубликаты
df.duplicated().sum()

697

In [20]:
# удаляем дубликаты
df = df.drop_duplicates()
df.duplicated().sum()

0

In [21]:
# получим список ссылок на сайты
all_car = df[0].tolist()

In [22]:
# запускаем сбор данных
df_train_voronezh = get_car_data(all_car)
df_train_voronezh

100%|██████████| 2529/2529 [46:49<00:00,  1.11s/it]  


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль,Состояние,Таможня,price,Владение
0,хэтчбек 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/civic/110...,чёрный,NaN,Автомобиль проверен и готов к эксплуатации!\n\...,1.8 LTR,140 N12,"{'cruise-control': True, 'esp': True, 'usb': T...",бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,160000,2005,"{'code': 'CIVIC', 'name': 'Civic', 'ru_name': ...",CIVIC,1.8 MT (140 л.с.),5,1636620913,RUB,2007,1105039647,"{'id': '2306769', 'name': '', 'nameplate': '',...",HATCHBACK_5_DOORS MECHANICAL 1.8,механическая,NaN,3 или более,Оригинал,передний,Левый,Не требует ремонта,Растаможен,500000.0,NaN
0,внедорожник 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/cr_v/1105...,коричневый,NaN,ОНЛАЙН-КОМИССИЯ\nВнимание! Осмотр автомобиля в...,2.0 LTR,150 N12,"{'cruise-control': True, 'asr': True, 'tinted-...",бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,167000,2011,"{'code': 'CR_V', 'name': 'CR-V', 'ru_name': 'C...",CR_V,2.0 AT (150 л.с.) 4WD,5,1636620915,RUB,2013,1105919645,"{'id': '8510734', 'name': '', 'nameplate': '',...",ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,NaN,2 владельца,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1740000.0,NaN
0,минивэн,HONDA,https://auto.ru/cars/used/sale/honda/stepwagon...,чёрный,NaN,Один из самых популярных и высокотехнологичных...,1.5 LTR,150 N12,"{'tinted-glass': True, 'esp': True, 'usb': Tru...",бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,58000,2015,"{'code': 'STEPWAGON', 'name': 'Stepwgn', 'ru_n...",STEPWAGON,Spada 1.5 CVT (150 л.с.) 4WD,5,1636620916,RUB,2016,1105404385,"{'id': '21125713', 'name': 'Spada', 'nameplate...",MINIVAN VARIATOR 1.5,вариатор,NaN,1 владелец,Оригинал,полный,Правый,Не требует ремонта,Растаможен,1730000.0,NaN
0,внедорожник 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/cr_v/1105...,чёрный,NaN,Автомобиль не требует вложений и полностью гот...,2.4 LTR,166 N12,"{'cruise-control': True, 'electro-window-back'...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,146000,2009,"{'code': 'CR_V', 'name': 'CR-V', 'ru_name': 'C...",CR_V,2.4 AT (166 л.с.) 4WD,5,1636620917,RUB,2010,1105682483,"{'id': '6312981', 'name': '', 'nameplate': '',...",ALLROAD_5_DOORS AUTOMATIC 2.4,автоматическая,NaN,3 или более,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1210000.0,NaN
0,хэтчбек 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/civic/110...,серебристый,NaN,авто в идеальном состоянии. После предыдущего ...,2.2 LTR,140 N12,"{'cruise-control': True, 'asr': True, 'esp': T...",дизель,https://avatars.mds.yandex.net/get-autoru-vos/...,157000,2008,"{'code': 'CIVIC', 'name': 'Civic', 'ru_name': ...",CIVIC,2.2d MT (140 л.с.),5,1636620918,RUB,2011,1105904871,"{'id': '7699571', 'name': '', 'nameplate': '',...",HATCHBACK_5_DOORS MECHANICAL 2.2,механическая,NaN,3 или более,Оригинал,передний,Левый,Не требует ремонта,Растаможен,830000.0,6 месяцев
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,компактвэн,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/spac...,серый,NaN,"Большой cемeйный автoмобиль, три ряда сидeний ...",2.4 LTR,147 N12,{},бензин,https://avatars.mds.yandex.net/get-verba/21620...,248000,1998,"{'code': 'SPACE_WAGON', 'name': 'Space Wagon',...",SPACE_WAGON,2.4 AT (147 л.с.) 4WD,5,1636623718,RUB,2003,1105465144,"{'id': '7761985', 'name': '', 'nameplate': '',...",COMPACTVAN AUTOMATIC 2.4,автоматическая,NaN,2 владельца,Дубликат,полный,Левый,Не требует ремонта,Растаможен,280000.0,NaN
0,хэтчбек 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/cari...,серый,NaN,"Продам личный автомобиль, покупал жене не зашё...",1.8 LTR,122 N12,{},бензин,https://autoru

In [23]:
# добавим признак города
df_train_voronezh['city'] = 'Воронеж'
df_train_voronezh

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль,Состояние,Таможня,price,Владение,city
0,хэтчбек 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/civic/110...,чёрный,NaN,Автомобиль проверен и готов к эксплуатации!\n\...,1.8 LTR,140 N12,"{'cruise-control': True, 'esp': True, 'usb': T...",бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,160000,2005,"{'code': 'CIVIC', 'name': 'Civic', 'ru_name': ...",CIVIC,1.8 MT (140 л.с.),5,1636620913,RUB,2007,1105039647,"{'id': '2306769', 'name': '', 'nameplate': '',...",HATCHBACK_5_DOORS MECHANICAL 1.8,механическая,NaN,3 или более,Оригинал,передний,Левый,Не требует ремонта,Растаможен,500000.0,NaN,Воронеж
0,внедорожник 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/cr_v/1105...,коричневый,NaN,ОНЛАЙН-КОМИССИЯ\nВнимание! Осмотр автомобиля в...,2.0 LTR,150 N12,"{'cruise-control': True, 'asr': True, 'tinted-...",бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,167000,2011,"{'code': 'CR_V', 'name': 'CR-V', 'ru_name': 'C...",CR_V,2.0 AT (150 л.с.) 4WD,5,1636620915,RUB,2013,1105919645,"{'id': '8510734', 'name': '', 'nameplate': '',...",ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,NaN,2 владельца,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1740000.0,NaN,Воронеж
0,минивэн,HONDA,https://auto.ru/cars/used/sale/honda/stepwagon...,чёрный,NaN,Один из самых популярных и высокотехнологичных...,1.5 LTR,150 N12,"{'tinted-glass': True, 'esp': True, 'usb': Tru...",бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,58000,2015,"{'code': 'STEPWAGON', 'name': 'Stepwgn', 'ru_n...",STEPWAGON,Spada 1.5 CVT (150 л.с.) 4WD,5,1636620916,RUB,2016,1105404385,"{'id': '21125713', 'name': 'Spada', 'nameplate...",MINIVAN VARIATOR 1.5,вариатор,NaN,1 владелец,Оригинал,полный,Правый,Не требует ремонта,Растаможен,1730000.0,NaN,Воронеж
0,внедорожник 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/cr_v/1105...,чёрный,NaN,Автомобиль не требует вложений и полностью гот...,2.4 LTR,166 N12,"{'cruise-control': True, 'electro-window-back'...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,146000,2009,"{'code': 'CR_V', 'name': 'CR-V', 'ru_name': 'C...",CR_V,2.4 AT (166 л.с.) 4WD,5,1636620917,RUB,2010,1105682483,"{'id': '6312981', 'name': '', 'nameplate': '',...",ALLROAD_5_DOORS AUTOMATIC 2.4,автоматическая,NaN,3 или более,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1210000.0,NaN,Воронеж
0,хэтчбек 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/civic/110...,серебристый,NaN,авто в идеальном состоянии. После предыдущего ...,2.2 LTR,140 N12,"{'cruise-control': True, 'asr': True, 'esp': T...",дизель,https://avatars.mds.yandex.net/get-autoru-vos/...,157000,2008,"{'code': 'CIVIC', 'name': 'Civic', 'ru_name': ...",CIVIC,2.2d MT (140 л.с.),5,1636620918,RUB,2011,1105904871,"{'id': '7699571', 'name': '', 'nameplate': '',...",HATCHBACK_5_DOORS MECHANICAL 2.2,механическая,NaN,3 или более,Оригинал,передний,Левый,Не требует ремонта,Растаможен,830000.0,6 месяцев,Воронеж
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,компактвэн,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/spac...,серый,NaN,"Большой cемeйный автoмобиль, три ряда сидeний ...",2.4 LTR,147 N12,{},бензин,https://avatars.mds.yandex.net/get-verba/21620...,248000,1998,"{'code': 'SPACE_WAGON', 'name': 'Space Wagon',...",SPACE_WAGON,2.4 AT (147 л.с.) 4WD,5,1636623718,RUB,2003,1105465144,"{'id': '7761985', 'name': '', 'nameplate': '',...",COMPACTVAN AUTOMATIC 2.4,автоматическая,NaN,2 владельца,Дубликат,полный,Левый,Не требует ремонта,Растаможен,280000.0,NaN,Воронеж
0,хэтчбек 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/cari...,серый,NaN,"Продам личный автомобиль, покупал 

In [24]:
# сохраним датафрейм
df_train_voronezh.to_csv('df_train_voronezh.csv')
data = pd.read_csv('df_train_voronezh.csv')
data

,Unnamed: 0,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль,Состояние,Таможня,price,Владение,city
0,0,хэтчбек 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/civic/110...,чёрный,NaN,Автомобиль проверен и готов к эксплуатации!\n\...,1.8 LTR,140 N12,"{'cruise-control': True, 'esp': True, 'usb': T...",бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,160000,2005,"{'code': 'CIVIC', 'name': 'Civic', 'ru_name': ...",CIVIC,1.8 MT (140 л.с.),5,1636620913,RUB,2007,1105039647,"{'id': '2306769', 'name': '', 'nameplate': '',...",HATCHBACK_5_DOORS MECHANICAL 1.8,механическая,NaN,3 или более,Оригинал,передний,Левый,Не требует ремонта,Растаможен,500000.0,NaN,Воронеж
1,0,внедорожник 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/cr_v/1105...,коричневый,NaN,ОНЛАЙН-КОМИССИЯ\nВнимание! Осмотр автомобиля в...,2.0 LTR,150 N12,"{'cruise-control': True, 'asr': True, 'tinted-...",бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,167000,2011,"{'code': 'CR_V', 'name': 'CR-V', 'ru_name': 'C...",CR_V,2.0 AT (150 л.с.) 4WD,5,1636620915,RUB,2013,1105919645,"{'id': '8510734', 'name': '', 'nameplate': '',...",ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,NaN,2 владельца,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1740000.0,NaN,Воронеж
2,0,минивэн,HONDA,https://auto.ru/cars/used/sale/honda/stepwagon...,чёрный,NaN,Один из самых популярных и высокотехнологичных...,1.5 LTR,150 N12,"{'tinted-glass': True, 'esp': True, 'usb': Tru...",бензин,https://autoru.naydex.net/AQZ98gb46/8b074fVQd7...,58000,2015,"{'code': 'STEPWAGON', 'name': 'Stepwgn', 'ru_n...",STEPWAGON,Spada 1.5 CVT (150 л.с.) 4WD,5,1636620916,RUB,2016,1105404385,"{'id': '21125713', 'name': 'Spada', 'nameplate...",MINIVAN VARIATOR 1.5,вариатор,NaN,1 владелец,Оригинал,полный,Правый,Не требует ремонта,Растаможен,1730000.0,NaN,Воронеж
3,0,внедорожник 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/cr_v/1105...,чёрный,NaN,Автомобиль не требует вложений и полностью гот...,2.4 LTR,166 N12,"{'cruise-control': True, 'electro-window-back'...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,146000,2009,"{'code': 'CR_V', 'name': 'CR-V', 'ru_name': 'C...",CR_V,2.4 AT (166 л.с.) 4WD,5,1636620917,RUB,2010,1105682483,"{'id': '6312981', 'name': '', 'nameplate': '',...",ALLROAD_5_DOORS AUTOMATIC 2.4,автоматическая,NaN,3 или более,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1210000.0,NaN,Воронеж
4,0,хэтчбек 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/civic/110...,серебристый,NaN,авто в идеальном состоянии. После предыдущего ...,2.2 LTR,140 N12,"{'cruise-control': True, 'asr': True, 'esp': T...",дизель,https://avatars.mds.yandex.net/get-autoru-vos/...,157000,2008,"{'code': 'CIVIC', 'name': 'Civic', 'ru_name': ...",CIVIC,2.2d MT (140 л.с.),5,1636620918,RUB,2011,1105904871,"{'id': '7699571', 'name': '', 'nameplate': '',...",HATCHBACK_5_DOORS MECHANICAL 2.2,механическая,NaN,3 или более,Оригинал,передний,Левый,Не требует ремонта,Растаможен,830000.0,6 месяцев,Воронеж
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2524,0,компактвэн,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/spac...,серый,NaN,"Большой cемeйный автoмобиль, три ряда сидeний ...",2.4 LTR,147 N12,{},бензин,https://avatars.mds.yandex.net/get-verba/21620...,248000,1998,"{'code': 'SPACE_WAGON', 'name': 'Space Wagon',...",SPACE_WAGON,2.4 AT (147 л.с.) 4WD,5,1636623718,RUB,2003,1105465144,"{'id': '7761985', 'name': '', 'nameplate': '',...",COMPACTVAN AUTOMATIC 2.4,автоматическая,NaN,2 владельца,Дубликат,полный,Левый,Не требует ремонта,Растаможен,280000.0,NaN,Воронеж
2525,0,хэтчбек 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/cari...,серый,NaN,

Парсинг выполнялся параллельно на нескольких ноутбуках Google Colab и Jupiter Notebook. Объединим полученные датафреймы.

In [2]:
df1 = pd.read_csv('df_train_moskva.csv', index_col=0)
df2 = pd.read_csv('df_train_sankt-peterburg.csv', index_col=0)
df3 = pd.read_csv('df_train_novosibirsk.csv', index_col=0)
df4 = pd.read_csv('df_train_ekaterinburg.csv', index_col=0)
df5 = pd.read_csv('df_train_kazan.csv', index_col=0)
df6 = pd.read_csv('df_train_nizhniy_novgorod.csv', index_col=0)
df7 = pd.read_csv('df_train_chelyabinsk.csv', index_col=0)
df8 = pd.read_csv('df_train_samara.csv', index_col=0)
df9 = pd.read_csv('df_train_omsk.csv', index_col=0)
df10 = pd.read_csv('df_train_rostov-na-donu.csv', index_col=0)
df11 = pd.read_csv('df_train_ufa.csv', index_col=0)
df12 = pd.read_csv('df_train_krasnoyarsk.csv', index_col=0)
df13 = pd.read_csv('df_train_voronezh.csv', index_col=0)
df14 = pd.read_csv('df_train_perm.csv', index_col=0)
df15 = pd.read_csv('df_train_volgograd.csv', index_col=0)

In [3]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15], ignore_index=True)
df

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price,city
0,седан,HONDA,https://auto.ru/cars/used/sale/honda/accord/11...,серебристый,NaN,"Машина в максимальной комплектации: кожа, люк,...",2.4 LTR,190 N12,"{'cruise-control': True, 'asr': True, 'tinted-...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,198781.0,2002.0,"{'code': 'ACCORD', 'name': 'Accord', 'ru_name'...",ACCORD,2.4 AT (190 л.с.),4.0,1636641004,RUB,2005.0,1.105910e+09,"{'id': '20495769', 'displacement': 2354, 'engi...",SEDAN AUTOMATIC 2.4,автоматическая,NaN,3 или более,1 год и 4 месяца,Оригинал,передний,Левый,Не требует ремонта,Растаможен,559400.0,Москва
1,седан,HONDA,https://auto.ru/cars/used/sale/honda/civic/110...,серебристый,NaN,Автомобиль был куплен новым у официального дил...,1.8 LTR,140 N12,"{'engine-proof': True, 'asr': True, 'tinted-gl...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,200000.0,2005.0,"{'code': 'CIVIC', 'name': 'Civic', 'ru_name': ...",CIVIC,1.8 MT (140 л.с.),4.0,1636641005,RUB,2008.0,1.105428e+09,"{'id': '2306778', 'displacement': 1798, 'engin...",SEDAN MECHANICAL 1.8,механическая,NaN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,464000.0,Москва
2,внедорожник 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/cr_v/1105...,коричневый,NaN,"Продаю семейный автомобиль, который служил всё...",2.0 LTR,150 N12,"{'electro-window-back': True, 'paint-metallic'...",бензин,https://autoru.naydex.net/Bzqs85G78/1a23943g/j...,106000.0,2009.0,"{'code': 'CR_V', 'name': 'CR-V', 'ru_name': 'C...",CR_V,2.0 MT (150 л.с.) 4WD,5.0,1636641007,RUB,2011.0,1.105765e+09,"{'id': '6312920', 'displacement': 1997, 'engin...",ALLROAD_5_DOORS MECHANICAL 2.0,механическая,NaN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1300000.0,Москва
3,внедорожник 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/hr_v/1105...,красный,NaN,В хорошем состоянии!На уверенном ходу!\nЮр.чис...,1.6 LTR,105 N12,"{'asr': True, 'tinted-glass': True, 'esp': Tru...",бензин,https://autoru.naydex.net/Bzqs85G78/1a23943g/j...,250000.0,2001.0,"{'code': 'HR_V', 'name': 'HR-V', 'ru_name': 'H...",HR_V,1.6 CVT (105 л.с.) 4WD,5.0,1636641008,RUB,2003.0,1.105847e+09,"{'id': '6471712', 'displacement': 1590, 'engin...",ALLROAD_5_DOORS VARIATOR 1.6,вариатор,NaN,3 или более,NaN,Дубликат,полный,Левый,Не требует ремонта,Растаможен,343000.0,Москва
4,седан,HONDA,https://auto.ru/cars/used/sale/honda/accord/11...,синий,NaN,Honda Accord 1998 года. 1 владелец. Двигатель ...,1.9 LTR,136 N12,{},бензин,https://autoru.naydex.net/Bzqs85G78/1a23943g/j...,300000.0,1997.0,"{'code': 'ACCORD', 'name': 'Accord', 'ru_name'...",ACCORD,1.9 AT (136 л.с.),4.0,1636641010,RUB,1998.0,1.105414e+09,"{'id': '6307020', 'displacement': 1849, 'engin...",SEDAN AUTOMATIC 1.9,автоматическая,NaN,1 владелец,10 лет и 9 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,220000.0,Москва
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83406,внедорожник 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/outl...,белый,NaN,ОНЛАЙН-КОМИССИЯ\nВнимание! Осмотр автомобиля в...,3.0 LTR,230 N12,"{'cruise-control': True, 'asr': True, 'tinted-...",бензин,https://autoru.naydex.net/V1zDN7454/d837e6v_2Y...,120000.0,2012.0,"{'code': 'OUTLANDER', 'name': 'Outlander', 'ru...",OUTLANDER,3.0 AT (230 л.с.) 4WD,5.0,1636696435,RUB,2013.0,1.104018e+09,"{'id': '10374150', 'displacement': 2998, 'engi...",ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,NaN,3 или более,NaN,Дубликат,полный,Левый,Не требует ремонта,Растаможен,1490000.0,Волгоград
83407,седан,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/lanc...,чёрный

In [4]:
# сохраняем датафрейм
df.to_csv('all_car_11_11_2021.csv')